## Day 1: Trebuchet?!

In [53]:
sample = """1abc2
pqr3stu8vwx
a1b2c3d4e5f
treb7uchet""".split('\n')
sample

['1abc2', 'pqr3stu8vwx', 'a1b2c3d4e5f', 'treb7uchet']

In [55]:
import re

def extract_calibration_value(txt, debug=False):
    digits = re.match(r'^.*?(\d)', txt).group(1), re.match(r'.*(\d).*$', txt).group(1)
    if debug:
        print(digits)
    return int(''.join(digits))

sample_res = [12, 38, 15, 77]
assert all(extract_calibration_value(line, debug=True) == res for line, res in zip(sample, sample_res))

('1', '2')
('3', '8')
('1', '5')
('7', '7')


In [41]:
with open('input_day1.txt') as f:
    lines = f.readlines()
len(lines), lines[-1]

(1000, 'mbnfjkxptbtjmgcrtkhxjvjhjnine83mpnsixfcmxcbnspx\n')

In [42]:
sum(map(extract_calibration_value, lines))

53974

In [61]:
sample = """two1nine
eightwothree
abcone2threexyz
xtwone3four
4nineeightseven2
zoneight234
7pqrstsixteen""".split('\n')
sample_res = [29, 83, 13, 24, 42, 14, 76]

DIGIT_STRINGS = ['one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine']
DIGIT_MAP = {s: str(i+1) for i, s in enumerate(DIGIT_STRINGS)}
DIGIT_REGEX = r'(\d|' + '|'.join(DIGIT_STRINGS) + ')'
print(DIGIT_REGEX)

def extract_calibration_value_2(txt, debug=False):
    digits = re.match(rf'^.*?{DIGIT_REGEX}', txt).group(1), re.match(rf'.*{DIGIT_REGEX}.*$', txt).group(1)
    digits = [DIGIT_MAP.get(d) or d for d in digits]
    if debug:
        print(digits)
    return int(''.join(digits))

assert all(extract_calibration_value_2(line, debug=True) == res for line, res in zip(sample, sample_res))

(\d|one|two|three|four|five|six|seven|eight|nine)
['2', '9']
['8', '3']
['1', '3']
['2', '4']
['4', '2']
['1', '4']
['7', '6']


In [62]:
sum(map(extract_calibration_value_2, lines))

52840

## Day 2: Cube Conundrum

In [63]:
def parse_game(line):
    game = [dict((lambda p: (p[1], int(p[0])))(chunk.split()) for chunk in draw.split(', '))
            for draw in line.strip().split('; ')]
    return game

l = '3 blue, 4 red; 1 red, 2 green, 6 blue; 2 green'
parse_game(l)

[{'blue': 3, 'red': 4}, {'red': 1, 'green': 2, 'blue': 6}, {'green': 2}]

In [64]:
sample = """Game 1: 3 blue, 4 red; 1 red, 2 green, 6 blue; 2 green
Game 2: 1 blue, 2 green; 3 green, 4 blue, 1 red; 1 green, 1 blue
Game 3: 8 green, 6 blue, 20 red; 5 blue, 4 red, 13 green; 5 green, 1 red
Game 4: 1 green, 3 red, 6 blue; 3 green, 6 red; 3 green, 15 blue, 14 red
Game 5: 6 red, 1 blue, 3 green; 2 blue, 1 red, 2 green""".split('\n')

In [65]:
def sum_possible_games(lines, max_vals={'red': 12, 'green': 13, 'blue': 14}, debug=False):
    res = 0
    for i, line in enumerate(lines):
        game = parse_game(line.split(': ')[1])
        if debug:
            print(game)
        if not any(any(draw[k] > max_vals[k] for k in draw)
                   for draw in game):
            res += i+1
    return res

sum_possible_games(sample, debug=True)

[{'blue': 3, 'red': 4}, {'red': 1, 'green': 2, 'blue': 6}, {'green': 2}]
[{'blue': 1, 'green': 2}, {'green': 3, 'blue': 4, 'red': 1}, {'green': 1, 'blue': 1}]
[{'green': 8, 'blue': 6, 'red': 20}, {'blue': 5, 'red': 4, 'green': 13}, {'green': 5, 'red': 1}]
[{'green': 1, 'red': 3, 'blue': 6}, {'green': 3, 'red': 6}, {'green': 3, 'blue': 15, 'red': 14}]
[{'red': 6, 'blue': 1, 'green': 3}, {'blue': 2, 'red': 1, 'green': 2}]


8

In [66]:
with open('input_day2.txt') as f:
    lines = f.readlines()
len(lines), lines[-1]

(100, 'Game 100: 10 red; 11 blue, 12 red; 1 green, 7 blue, 6 red\n')

In [67]:
sum_possible_games(lines)

2679

In [71]:
from collections import defaultdict
from math import prod


def compute_game_power(game):
    min_vals = defaultdict(int)
    for draw in game:
        for k in draw:
            min_vals[k] = max(draw[k], min_vals[k])
    return prod(min_vals.values())
    
powers = [48, 12, 1560, 630, 36]
assert all(compute_game_power(parse_game(line.split(': ')[1])) == power
           for line, power in zip(sample, powers))

In [72]:
sum(compute_game_power(parse_game(line.split(': ')[1]))
    for line in lines)

77607

## Day 3: Gear Ratios

In [80]:
sample = """
467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598..""".strip().split('\n')

In [97]:
import re
m = list(re.finditer(r'(\d+)', sample[0]))[0]
m.start(), m.end(), int(m.group())

(0, 3, 467)

In [143]:
def extract_part_numbers(lines, debug=False):
    last_numbers, last_symbols = [], set()
    part_numbers = []
    for i, line in enumerate(lines):
        numbers = [(m.start(), m.end(), int(m.group())) for m in re.finditer(r'(\d+)', line)]
        symbols = list(re.finditer(r'([^\d.])', line.strip()))
        if debug:
            print(numbers, [s.group() for s in symbols])
        symbols = set(m.start() for m in symbols)

        new_nums = []
        for start, end, n in numbers:
            if any(abs(start - 1) <= s <= min(len(line), end) for s in symbols | last_symbols):
                new_nums.append(n)
        for start, end, n in last_numbers:
            if any(abs(start - 1) <= s <= min(len(line), end) for s in symbols):
                new_nums.append(n)
        
        if debug:
            print(f'#{i} Part numbers <- {new_nums}\n')

        part_numbers += new_nums
        last_numbers, last_symbols = numbers, symbols
    return part_numbers

assert sum(extract_part_numbers(sample, debug=True)) == 4361

[(0, 3, 467), (5, 8, 114)] []
#0 Part numbers <- []

[] ['*']
#1 Part numbers <- [467]

[(2, 4, 35), (6, 9, 633)] []
#2 Part numbers <- [35]

[] ['#']
#3 Part numbers <- [633]

[(0, 3, 617)] ['*']
#4 Part numbers <- [617]

[(7, 9, 58)] ['+']
#5 Part numbers <- []

[(2, 5, 592)] []
#6 Part numbers <- [592]

[(6, 9, 755)] []
#7 Part numbers <- []

[] ['$', '*']
#8 Part numbers <- [755]

[(1, 4, 664), (5, 8, 598)] []
#9 Part numbers <- [664, 598]



In [144]:
with open('input_day3.txt') as f:
    lines = f.readlines()
len(lines), lines[-1].strip()

(140,
 '.......................................261......807.261.............................42......................................................')

In [145]:
sum(extract_part_numbers(lines))

522726